# Setup

In [1]:
%load_ext autoreload

In [3]:
%autoreload

import my_functions as mf
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.figsize'] = (10, 5)
plt.style.use("seaborn-v0_8-darkgrid")
import seaborn as sns
sns.set(rc={"figure.dpi":200, 'savefig.dpi':200 , 'figure.figsize': (10, 5)})
sns.set_context('notebook')
sns.set_style("ticks")
sns.set_style('darkgrid')

from pycaret.regression import *

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML

In [7]:
ignore_features = [
    "ceiling_height_agl:m",
    "cloud_base_agl:m",
    "snow_density:kgm3",
]

In [8]:
j_Y_train_a = pd.read_parquet("Jørgen_cleaned_data/y_train_a.parquet")
j_Y_train_a.drop(["time"], axis=1, inplace=True)
j_Y_train_b = pd.read_parquet("Jørgen_cleaned_data/y_train_b.parquet")
j_Y_train_b.drop(["time"], axis=1, inplace=True)
j_Y_train_c = pd.read_parquet("Jørgen_cleaned_data/y_train_c.parquet")
j_Y_train_c.drop(["time"], axis=1, inplace=True)
j_Y_train = pd.concat([j_Y_train_a, j_Y_train_b, j_Y_train_c], ignore_index=True)

j_X_train_a = pd.read_parquet("Jørgen_cleaned_data/x_train_a.parquet")
j_X_train_b = pd.read_parquet("Jørgen_cleaned_data/x_train_b.parquet")
j_X_train_c = pd.read_parquet("Jørgen_cleaned_data/x_train_c.parquet")
j_X_train = pd.concat([j_X_train_a, j_X_train_b, j_X_train_c], ignore_index=True)

j_train_a = pd.concat([j_Y_train_a, j_X_train_a], axis = 1)
j_train_b = pd.concat([j_Y_train_b, j_X_train_b], axis = 1)
j_train_c = pd.concat([j_Y_train_c, j_X_train_c], axis = 1)

j_X_test_a = pd.read_parquet("Jørgen_cleaned_data/x_test_a.parquet")
j_X_test_b = pd.read_parquet("Jørgen_cleaned_data/x_test_b.parquet")
j_X_test_c = pd.read_parquet("Jørgen_cleaned_data/x_test_c.parquet")
j_X_test = pd.concat([j_X_test_a, j_X_test_b, j_X_test_c], ignore_index=True)

# Main stuff:

In [19]:
j_reg_full_a = RegressionExperiment()
j_reg_full_a.setup(
    data = j_train_a,
    use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    low_variance_threshold = 0,
    fold = 25,
    imputation_type="iterative",
    iterative_imputation_iters=25,
    # feature_selection=True,
    # n_features_to_select = 32,
    # pca = True,
    # html = False,
    session_id = 97,
    )

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are n

,Description,Value
0,Session id,97
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(18036, 163)"
4,Transformed data shape,"(18036, 160)"
5,Transformed train set shape,"(18035, 160)"
6,Transformed test set shape,"(1, 160)"
7,Numeric features,162
8,Rows with missing values,98.5%
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are n

In [17]:
# j_reg_full_a.compare_models(fold = 2)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,323.3617,276498.6364,525.8312,0.8545,0.8931,3.0470,16.0400
lightgbm,Light Gradient Boosting Machine,324.3885,277725.5992,526.9901,0.8538,0.8631,2.7245,7.3350
et,Extra Trees Regressor,327.6833,286973.5490,535.6977,0.8490,0.7144,1.4027,8.1550
xgboost,Extreme Gradient Boosting,336.6868,294213.8906,542.4019,0.8451,0.9679,4.1640,6.7300
rf,Random Forest Regressor,338.0325,304692.2032,551.9869,0.8396,0.7313,1.4708,15.6150
gbr,Gradient Boosting Regressor,365.8775,332992.1501,577.0523,0.8247,1.0344,4.8906,29.6200
ridge,Ridge Regression,411.5619,360570.0539,600.4695,0.8102,1.3122,12.1742,6.1350
lr,Linear Regression,412.3228,361668.9158,601.3831,0.8097,1.3094,12.0528,6.0050
llar,Lasso Least Angle Regression,410.7327,362155.6862,601.7875,0.8094,1.2976,11.8406,5.7700
lasso,Lasso Regression,413.6596,364633.1091,603.8371,0.8081,1.3115,12.6596,6.8650


In [29]:
j_lightgbm_full_a = j_reg_full_a.create_model(
    estimator = "lightgbm",
    fold = 2,
    )
j_catboost_full_a = j_reg_full_a.create_model(
    estimator = "catboost",
    fold = 2,
    )
j_extra_trees_full_a = j_reg_full_a.create_model(
    estimator = "et",
    fold = 2,
    )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,328.2793,283404.0904,532.3571,0.8519,0.8492,2.6876
1,323.1778,275028.3506,524.4315,0.8542,0.8723,2.7100
Mean,325.7285,279216.2205,528.3943,0.8531,0.8607,2.6988
Std,2.5508,4187.8699,3.9628,0.0011,0.0116,0.0112


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,324.2127,275915.6646,525.2768,0.8559,0.8916,3.1696
1,322.3938,275800.7876,525.1674,0.8538,0.8897,3.0066
Mean,323.3033,275858.2261,525.2221,0.8548,0.8906,3.0881
Std,0.9095,57.4385,0.0547,0.0010,0.0010,0.0815


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,329.4322,289485.8355,538.0389,0.8488,0.7060,1.3096
1,326.1278,284905.3048,533.7652,0.8489,0.7215,1.4810
Mean,327.7800,287195.5701,535.9020,0.8489,0.7138,1.3953
Std,1.6522,2290.2653,2.1368,0.0001,0.0077,0.0857


In [31]:
j_blended_lightgbm_catboost_extra_trees_full_a = j_reg_full_a.blend_models(
    estimator_list = [
        j_lightgbm_full_a,
        j_catboost_full_a,
        # j_extra_trees_full_a,
        ],
    optimize = "MAE",
    fold = 2,
    )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,320.9688,272493.4055,522.0090,0.8576,0.8267,2.4073
1,317.5993,269831.2793,519.4529,0.8569,0.8414,2.5482
Mean,319.2840,271162.3424,520.7309,0.8573,0.8340,2.4777
Std,1.6847,1331.0631,1.2781,0.0004,0.0074,0.0704


In [30]:
j_blended_lightgbm_catboost_extra_trees_full_a = j_reg_full_a.blend_models(
    estimator_list = [
        j_lightgbm_full_a,
        j_catboost_full_a,
        j_extra_trees_full_a,
        ],
    optimize = "MAE",
    fold = 2,
    )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,319.9664,272632.5064,522.1422,0.8576,0.7754,1.9942
1,315.7020,268288.5878,517.9658,0.8578,0.7969,2.1845
Mean,317.8342,270460.5471,520.0540,0.8577,0.7862,2.0894
Std,2.1322,2171.9593,2.0882,0.0001,0.0108,0.0951
